In [1]:
# pip install scrapy
# pip install --upgrade zope2

from collections import Counter
import foursquare
import json
import matplotlib.pyplot as plt
import numpy as np
import operator
import pandas as pd
import pickle
import random
import requests
from scrapy import Selector
from scrapy.http import HtmlResponse
import seaborn as sns
from sqlalchemy import create_engine
import sqlite3
import textacy
import textacy.data
import time
import unicodedata

%matplotlib inline
%config InlineBackend.figure_format = 'retina'

In [2]:
usda = pd.read_pickle('../../../../Accounts/Electo:Computad/\
MacBook Pro/Software/Github/Old_repos/DSI-SF-4-anotherguitar/projects/Capstone Stuff/usda.pkl')
food = pd.read_pickle('../../../../Accounts/Electo:Computad/\
MacBook Pro/Software/Github/Old_repos/DSI-SF-4-anotherguitar/projects/Capstone Stuff/explore_endpoint_df.pkl')
vegan = food[food.vegan_venue_check == 'vegan']

In [3]:
def word_counter(unique_desc):
    single_string = ' '.join(unique_desc)
    count_string = len(single_string.split())
    return count_string

In [4]:
# have to count the number of words in each list and add this as a new column to the dataframe
updated_food = food
updated_food['count_desc_list'] = food.venue_desc_list.map(lambda x: word_counter(x))

In [5]:
def item_desc_counter(phrase):
    if phrase == 'desc_not_available':
        return 0
    else:
        countered = len(phrase.split())
        return countered

In [6]:
updated_food['count_item_name'] = food.menu_item_name.map(lambda x: len(x.split()))
updated_food['count_item_desc'] = food.menu_item_desc.map(lambda x: item_desc_counter(x))

In [7]:
updated_vegan = updated_food[updated_food.vegan_venue_check == 'vegan']

In [8]:
usda = usda.drop('ComName', axis=1)
usda = usda.drop('FG_Cd', axis=1)

In [9]:
usda.head()

,Food_No,Food_Group,Description,Nutr_No,Nutrient,Amount_per_100,Units,Tagname,Not_Imputed
0,01001,Dairy and Egg Products,"Butter, salted",203,Protein,0.85,g,PROCNT,1
1,01001,Dairy and Egg Products,"Butter, salted",204,Total lipid (fat),81.11,g,FAT,1
2,01001,Dairy and Egg Products,"Butter, salted",205,"Carbohydrate, by difference",0.06,g,CHOCDF,0
3,01001,Dairy and Egg Products,"Butter, salted",207,Ash,2.11,g,ASH,1
4,01001,Dairy and Egg Products,"Butter, salted",208,Energy,717.00,kcal,ENERC_KCAL,0


In [10]:
dads = usda[['Food_Group','Description']]

dads.Food_Group.unique()

# Columns I may drop/rationale:
# Probably nix Baby Foods (just baby versions, no unique identifier, so doesn't have predictive value)
# Probably nix Restaurant Foods (just restaurant versions, with some brand names, so doesn't have predictive value)
# Probably nix Breakfast Cereals - primarily brand names, w/ some 'cream' - nothing else helpful
# American Indian/Alaska Native Foods because it has a lot of fish, and is confusing my categories (I think)
# I found 'cream' in Soups, Sauces, and Gravies, but pultry products come up a lot too - check what id's come
# out of dairy

# Notes:
# On the fence: maybe nix Fast Foods, because it tends to be less informative (also the meals, entrees one)
# On the fence: Spices & Herbs (has some unique ingredients, but also generic words)
# Fats and Oils includes Mayo, make a unique one for mayo - is spelled as mayo, or full mayoneise on unique menus
#          ? b/c i think it becomes egg then, also fish oil, soy,
#       margarine is basically vegan except trace stuff, then there's butter too
# Snacks: can help on chips, but lots of brands, and doubles on milk, cheese, ingredient (does show granola too)
# Some Vegetables and Vegetable Products have milk b/c of processing, but fairly uncommon
# Nut and Seed Products - some oil mentions in there could throw it off, but otherwise decent; includes nut butters
# Finfish and Shellfish Products has raw, and so does Fruits, cut out? but could help w/ raw diets...
# Baked Products can include cheese, less of milk
# Sweets have egg, milk, honey, ice cream
# Cereal Grains and Pasta - some has egg
# Fast Foods has a lot of dairy, cheese, meats, pizza
# Meals, Entrees, and Sidedishes seems to have a lot of non-vegan stuff in general
# Restaurant Foods seems to have a lot of nonvegan and ethnic shit

array(['Dairy and Egg Products', 'Spices and Herbs', 'Baby Foods',
       'Fats and Oils', 'Poultry Products', 'Soups, Sauces, and Gravies',
       'Sausages and Luncheon Meats', 'Breakfast Cereals', 'Snacks',
       'Fruits and Fruit Juices', 'Pork Products',
       'Vegetables and Vegetable Products', 'Nut and Seed Products',
       'Beef Products', 'Beverages', 'Finfish and Shellfish Products',
       'Legumes and Legume Products', 'Lamb, Veal, and Game Products',
       'Baked Products', 'Sweets', 'Cereal Grains and Pasta', 'Fast Foods',
       'Meals, Entrees, and Sidedishes',
       'American Indian/Alaska Native Foods', 'Restaurant Foods'], dtype=object)

In [11]:
# Let's make a dictionary with the 25 unique food groups and their respective descriptions
deduped = []
for group, group_df in dads.groupby(['Food_Group', 'Description']):
    deduped.append({
      "Food_Group":  group[0],
      "Description": group[1]
            #imagine that I were able to grab the number of descriptions per food group here, it'd be redundant
            #here, but avoid the len check
    })

In [12]:
# This dictionary will give me the number of unique descriptions per food group
original_fg = {}
for classified in dads.Food_Group.unique():
    counter = 0
    for i in range(len(deduped)):
#   doesnt do anything
        if deduped[i]['Food_Group'] == classified:
            counter += 1
    original_fg[classified] = counter
# original_fg

In [13]:
number_dic = {
 'American Indian/Alaska Native Foods': range(0,165),
 'Baby Foods': range(165,498),
 'Baked Products': range(498, 1283),
 'Beef Products': range(1283, 2068),
 'Beverages': range(2068,2347),
 'Breakfast Cereals': range(2347,2719),
 'Cereal Grains and Pasta': range(2719,2903),
 'Dairy and Egg Products': range(2903,3157),
 'Fast Foods': range(3157,3552),
 'Fats and Oils': range(3552,3772),
 'Finfish and Shellfish Products': range(3772,4038),
 'Fruits and Fruit Juices': range(4038,4365),
 'Lamb, Veal, and Game Products': range(4365,4718),
 'Legumes and Legume Products': range(4718,5096),
 'Meals, Entrees, and Sidedishes': range(5096,5191),
 'Nut and Seed Products': range(5191,5320),
 'Pork Products': range(5320,5661),
 'Poultry Products': range(5661,5998),
 'Restaurant Foods': range(5998,6053),
 'Sausages and Luncheon Meats': range(6053,6287),
 'Snacks': range(6287,6458),
 'Soups, Sauces, and Gravies': range(6458,6973),
 'Spices and Herbs': range(6973,7034),
 'Sweets': range(7034,7378),
 'Vegetables and Vegetable Products': range(7378,8194)
}

In [14]:
fg_vallist_dic = {}
for classified in dads.Food_Group.unique():
    desc_list = []
    for item_i in number_dic[classified]:
        item_desc = deduped[item_i]['Description']
    
        desc_list.append(item_desc)
    
    fg_vallist_dic[classified] = desc_list
# fg_vallist_dic['Sweets'][343]

In [15]:
# Though that's okay, I'd prefer a dictionary with the number of words in each unique description per food group -
# this will help my proportion score down below surface even more relevant results
next_fg = {}
for classified in dads.Food_Group.unique():
    string_counter = 0
    for desc_string in usda[usda.Food_Group == classified].Description.unique():
        string_counter += len(desc_string.split())
    next_fg[classified] = string_counter
# next_fg

In [16]:
# Able to update as needed for experimentation
tmp_banned_cat = [
    'American Indian/Alaska Native Foods',
    'Baby Foods',
#     'Baked Products',
#     'Beef Products',
#     'Beverages',
#     'Breakfast Cereals',
#     'Cereal Grains and Pasta',
#     'Dairy and Egg Products',
    'Fast Foods',
#     'Fats and Oils',
#     'Finfish and Shellfish Products',
#     'Fruits and Fruit Juices',
#     'Lamb, Veal, and Game Products',
#     'Legumes and Legume Products',
    'Meals, Entrees, and Sidedishes',
#     'Nut and Seed Products',
#     'Pork Products',
#     'Poultry Products',
    'Restaurant Foods',
#     'Sausages and Luncheon Meats',
#     'Snacks',
#     'Soups, Sauces, and Gravies',
#     'Spices and Herbs',
#     'Sweets',
#     'Vegetables and Vegetable Products'
    ]

In [17]:
def assign_groups(word_in_item):
    fg_dic = {}
    
#     start_time = time.time()

    for grouped in dads.Food_Group.unique():
        if grouped not in tmp_banned_cat:
            counter_the = 0
            for item_i in range(original_fg[grouped]):
                search_this = fg_vallist_dic[grouped][item_i].lower().replace(',','').replace('/',' ').replace('-',' ').replace('_',' ').replace('.',' ').replace('(','').replace(')','').replace("'","").replace('&','')            
                if search_this.find(word_in_item.lower()) != -1:
                    counter_the += 1
                
            fg_dic[grouped] = float(counter_the) / next_fg[grouped]
#     Here is where I am improving my proportion with the ever-more accurate string counter dictionary
    
#     # Return the top [0-3] groups, but only if keyword exists in [0-3] groups
    max_groups_list = []
    if Counter(fg_dic).most_common(3)[0][1] != 0.0:
        max_groups_list.append(Counter(fg_dic).most_common(3)[0][0].replace(',',''))
        if Counter(fg_dic).most_common(3)[1][1] != 0.0:
            if Counter(fg_dic).most_common(3)[2][1] != 0.0:
                max_groups_list.append(Counter(fg_dic).most_common(3)[1][0].replace(',',''))
                max_groups_list.append(Counter(fg_dic).most_common(3)[2][0].replace(',',''))
            else:
                max_groups_list.append(Counter(fg_dic).most_common(3)[1][0].replace(',',''))
    
    if max_groups_list != []:
        max_groups_str = ', '.join(max_groups_list)
        return Counter(fg_dic).most_common(3)
    
# , ("-5- %s seconds ---" % (time.time() - start_time)), '\n-----------------------\n'

In [18]:
assign_groups('glably glab')

In [19]:
# mastery = list(assign_groups('salty')) + list(assign_groups('fish') + list(assign_groups('chicken') + list(assign_groups('fried') + list(assign_groups('rice')
# mastery

In [20]:
# mastery_dict = {}
# for t in mastery:
#     if t[0] in mastery_dict:
#         mastery_dict[t[0]] = mastery_dict[t[0]]+t[1]
#     else:
#         mastery_dict[t[0]] = t[1]
# mastery_dict

In [21]:
# This function will assign food groups if given a feature from a particular item as an input

def groups_from_item(itemed):

    #clean item
    item = itemed.lower().replace(',','').replace('/',' ').replace('-',' ').replace('_',' ').replace('.',' ').replace('(','').replace(')','').replace("'","").replace('&','')
    
    # create a placeholder (or a list) with all the applicable food groups for a given food item    
    all_food_groups = 'food_groups_placeholder'

    if len(item.split()) == 1:
#     because there's just one word, we can feed the item directly into the search loop
        word_in_item = item
        check_for_result = assign_groups(word_in_item)
        if check_for_result != None:
            all_food_groups = check_for_result

#         else:
#             all_food_groups = None
#             return all_food_groups

#     this is how I'll map the series with the correct menu item on the larger dataframe,
#     just in case I decide not to use the series map function directly

    else: # len(item.split()) >= 1:
        
        # because there's more than one word, I'll have to do an extra loop to hit each word, but making sure that
        # they get appended to the same row, and not to the following row
        
        each_words_group = []

        for word_per_item in item.split():
            mini_stop_words = ['and','with','by','for']            
            if word_per_item not in mini_stop_words:
                a_food_group = assign_groups(word_per_item)
            if a_food_group != None:
                each_words_group += list(a_food_group)

        if each_words_group != []:
            all_food_groups = each_words_group
            
#         else:
#             all_food_groups = 'no_groups_avail'
# #             return all_food_groups
            
#     all_food_groups needs to be converted into either the result of one, or the output
    
    max_groups_list = []    
    maxed_dic = {}

    if all_food_groups != None:
        try:
            for group in all_food_groups:
                if group[0] in maxed_dic:
                    maxed_dic[group[0]] = maxed_dic[group[0]]+group[1]
                elif group != None:
                    maxed_dic[group[0]] = group[1]

            rev_max_dic = sorted(maxed_dic.items(), key=lambda x: x[1])
            if rev_max_dic[-1][1] != 0.0:
                max_groups_list.append(rev_max_dic[-1][0].replace(',',''))
                if rev_max_dic[-2][1] != 0.0:
                    if rev_max_dic[-3][1] != 0.0:
                        max_groups_list.append(rev_max_dic[-2][0].replace(',',''))
                        max_groups_list.append(rev_max_dic[-3][0].replace(',',''))
                    else:
                        max_groups_list.append(rev_max_dic[-2][0].replace(',',''))

            if max_groups_list != []:
                max_groups_str = ', '.join(max_groups_list)
                return max_groups_str
        except:
            pass

        #rev_max_dic

    #     , ("--- %s seconds ---" % (time.time() - start_time))
        
# I could just create a list of possible food_groups first, then determine which food group is more likely? 
# for example, by taking the most common food group chosen for each of the words in that particular menu item

# ^^ I think the best way to do this is to make fg columns for item, menu, sub_menu, etc. and then pick the
# group label with the highest label, or maybe the top 2/3 labels per row

In [22]:
len(assign_groups('taco'))

3

In [23]:
groups_from_item("tofu")

'Legumes and Legume Products, Fats and Oils'

In [24]:
groups_from_item('salty fish and chicken fried rice')

'Finfish and Shellfish Products, Poultry Products, Cereal Grains and Pasta'

In [25]:
from ipywidgets import widgets
from IPython.display import display
search_food = widgets.Text(description="Search:", placeholder="Enter a food or ingredient!")
my_button = widgets.Button(description="This best be good...")

def on_submit(ui):
    print 'hey, i dont like it that rough'
#     print mytext.value
    
    to_predict = [(assign_groups(search_food.value))]
    print to_predict

my_button.on_click(on_submit)

display(search_food)
display(my_button)

In [26]:
list_run_thru = updated_vegan.menu_item_name
print len(list_run_thru), len(updated_food.menu_item_name)

1861 90047


In [27]:
# updated_food.menu_item_name

In [28]:
#  str(vegan_next_version.menu_item_name.values[0])
start_time = time.time()
print groups_from_item('Salty Fish and Chicken Fried Rice'), ("--- %s seconds ---" % (time.time() - start_time))

Finfish and Shellfish Products, Poultry Products, Cereal Grains and Pasta --- 0.399912118912 seconds ---


In [674]:
# Here I've automated the food group tagging I would have done across each item, sub menu, and meta menu
start_time = time.time()
updated_food['item_fg'] = updated_food.menu_item_name.map(lambda x: groups_from_item(x))
print("--- %s seconds ---" % (time.time() - start_time))

--- 428.829463005 seconds ---


/anaconda/envs/dsi/lib/python2.7/site-packages/ipykernel/__main__.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  app.launch_new_instance()


In [675]:
# updated_food.item_fg.value_counts()

Poultry Products, Soups Sauces and Gravies, Sausages and Luncheon Meats                       61
Beverages                                                                                     30
Vegetables and Vegetable Products                                                             25
Cereal Grains and Pasta, Snacks, Breakfast Cereals                                            20
Legumes and Legume Products, Vegetables and Vegetable Products, Fats and Oils                 20
Legumes and Legume Products                                                                   18
Beef Products, Lamb Veal and Game Products, Beverages                                         15
Pork Products, Spices and Herbs, Lamb Veal and Game Products                                  14
Sausages and Luncheon Meats, Beef Products, Soups Sauces and Gravies                          14
Vegetables and Vegetable Products, Fruits and Fruit Juices, Finfish and Shellfish Products    14
Spices and Herbs, Cereal Grain

In [229]:
updated_food['depth_menu_fg'] = updated_food.depth_menus_n.map(lambda x: groups_from_item(x))

/anaconda/envs/dsi/lib/python2.7/site-packages/ipykernel/__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


In [230]:
updated_food['meta_menu_fg'] = updated_food.meta_menu_n.map(lambda x: groups_from_item(x))

/anaconda/envs/dsi/lib/python2.7/site-packages/ipykernel/__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


In [231]:
updated_food['item_desc_fg'] = updated_food.menu_item_desc.map(
    lambda x: groups_from_item(x) if x != 'desc_not_available' else "no_groups_avail" )

/anaconda/envs/dsi/lib/python2.7/site-packages/ipykernel/__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app


In [232]:
updated_food[['venue_name','venue_rated','menu_item_name','menu_item_price','depth_menus_n','meta_menu_n','item_fg','depth_menu_fg','meta_menu_fg']].head(1)

,venue_name,venue_rated,menu_item_name,menu_item_price,depth_menus_n,meta_menu_n,item_fg,depth_menu_fg,meta_menu_fg
1064,Amber India,8.6,"Tokaji, Disznoko, Kesoi Szuret, Hungary '08",9,"Ports, Madeira, & Dessert Wines",Dessert Menu,no_groups_avail,"Beverages, Beverages",Beverages


In [233]:
updated_food[['menu_item_name','menu_item_desc','item_fg','depth_menu_fg','meta_menu_fg','item_desc_fg']].head(5)

,menu_item_name,menu_item_desc,item_fg,depth_menu_fg,meta_menu_fg,item_desc_fg
1064,"Tokaji, Disznoko, Kesoi Szuret, Hungary '08",desc_not_available,no_groups_avail,"Beverages, Beverages",Beverages,no_groups_avail
1065,"Tawny Port, Graham's 20 Yr",desc_not_available,"Baked Products, Baked Products, Beef Products,...","Beverages, Beverages",Beverages,no_groups_avail
1066,"Porto, Rozes, 1997",desc_not_available,no_groups_avail,"Beverages, Beverages",Beverages,no_groups_avail
1067,"Porto, Black, Quinta Do Noval, Portugal",desc_not_available,"Beverages, Baked Products","Beverages, Beverages",Beverages,no_groups_avail
1068,"Muscat, Casta Diva, Alicante Do, Spain '07",desc_not_available,"Beverages, Baked Products, Fast Foods","Beverages, Beverages",Beverages,no_groups_avail


In [ ]:
updated_food.to_pickle('../../../../Accounts/Electo:Computad/\
MacBook Pro/Software/Github/Old_repos/DSI-SF-4-anotherguitar/projects/Capstone Stuff/updated_food.pkl')
updated_food = pd.read_pickle('../../../../Accounts/Electo:Computad/\
MacBook Pro/Software/Github/Old_repos/DSI-SF-4-anotherguitar/projects/Capstone Stuff/updated_food.pkl')

In [ ]:
updated_vegan = updated_food[updated_food.vegan_venue_check == 'vegan']

In [194]:
zz = updated_vegan[updated_vegan.menu_item_name == 'Lemon Grass Panna Cotta']
zz.shape

(1, 14)

In [247]:
im = updated_vegan[updated_vegan.menu_item_name == 'Lemon Grass Panna Cotta']['item_fg']
dm = updated_vegan[updated_vegan.menu_item_name == 'Lemon Grass Panna Cotta']['depth_menu_fg']
mm = updated_vegan[updated_vegan.menu_item_name == 'Lemon Grass Panna Cotta']['meta_menu_fg']
idm = updated_vegan[updated_vegan.menu_item_name == 'Lemon Grass Panna Cotta']['item_desc_fg']
mm = updated_vegan[updated_vegan.menu_item_name == 'Lemon Grass Panna Cotta']

# print vegan.map(lambda x: 

for i in range(updated_vegan.shape[0]):
    collect_descs = []
    
    # The iloc collects the index of each description I will tag with a group name
    # I'm taking the outputs of my previous function on columns 5,6,7 & 9, which are in 10,11,12, & 13

    try:
        for j in updated_vegan.iloc[i,[10,11,12,13]]:
            collect_descs.append(j)
            collected = ', '.join(collect_descs)
        collected = str(collected)
        collected = [y for y in collected.split(', ')]
        top_descs = Counter(collected).most_common(3)
        x = top_descs
        print "\n----\n", updated_vegan.iloc[i,[7]], x
        try:
            print x[0][0], x[0][1]
        except:
            pass
        try:
            print x[1][0], x[1][1]
        except:
            pass
        try:
            print x[2][0], x[2][1]
        except:
            pass
    except:
        pass

# im = vegan_next_version[vegan_next_version.menu_item_name == 'Pop Chips']['item_fg']
# dm = vegan_next_version[vegan_next_version.menu_item_name == 'Pop Chips']['depth_menu_fg']
# mm = vegan_next_version[vegan_next_version.menu_item_name == 'Pop Chips']['meta_menu_fg']
# idm = vegan_next_version[vegan_next_version.menu_item_name == 'Pop Chips']['item_desc_fg']

# def auto_tagger(row_example):
    
# for i in im:
#     for j in dm:
#         for k in mm:
#             for l in idm:
#                 listed = i, j, k, l
#                 listed = ', '.join(listed)
#                 listed = str(listed)
#                 listed = [y for y in listed.split(', ')]
#                 x = Counter(listed).most_common(3)
# print x[0][0], x[0][1]

# print x[1][0], x[1][1]

# print x[2][0], x[2][1]



----
menu_item_name    Tokaji, Disznoko, Kesoi Szuret, Hungary '08
Name: 1064, dtype: object [('Beverages', 3), ('no_groups_avail', 2)]
Beverages 3
no_groups_avail 2

----
menu_item_name    Tawny Port, Graham's 20 Yr
Name: 1065, dtype: object [('Beverages', 4), ('Baked Products', 2), ('no_groups_avail', 1)]
Beverages 4
Baked Products 2
no_groups_avail 1

----
menu_item_name    Porto, Rozes, 1997
Name: 1066, dtype: object [('Beverages', 3), ('no_groups_avail', 2)]
Beverages 3
no_groups_avail 2

----
menu_item_name    Porto, Black, Quinta Do Noval, Portugal
Name: 1067, dtype: object [('Beverages', 4), ('Baked Products', 1), ('no_groups_avail', 1)]
Beverages 4
Baked Products 1
no_groups_avail 1

----
menu_item_name    Muscat, Casta Diva, Alicante Do, Spain '07
Name: 1068, dtype: object [('Beverages', 4), ('Fast Foods', 1), ('Baked Products', 1)]
Beverages 4
Fast Foods 1
Baked Products 1

----
menu_item_name    Madeira, Blandy's 5 Yr, Alvada
Name: 1069, dtype: object [('Beverages', 4), ('

In [539]:
# happy = "Baked Products, Baked Products, Dairy and Egg Products, Baby Foods, Restaurant Foods, Baby Foods, Restaurant Foods, Baked Products, Baked Products, Baby Foods, American Indian/Alaska Native Foods, Vegetables and Vegetable Products, Baby Foods"
# happy.split(', ')
# for i in happy.split(', '):
#     print "item:", i

In [24]:
# Below is extra data that I might later use to correlate nutrients with particular food groups,
# though I'm not sure if there will be enough time for me to complete this aspect of the project

In [25]:
usda.shape

(595359, 9)

In [26]:
tmp = usda[['Food_Group','Nutrient']]

In [27]:
tmp = tmp.groupby(['Food_Group','Nutrient']).agg(lambda x: 1.)
tmp.head()

Food_Group                           Nutrient
American Indian/Alaska Native Foods  10:0        1.0
                                     12:0        1.0
                                     14:0        1.0
                                     14:1        1.0
                                     15:0        1.0
dtype: float64

In [28]:
tmp = tmp.unstack(fill_value=0)

In [29]:
tmp.shape

(25, 145)

In [30]:
tmp = tmp.reset_index()

In [31]:
tmp['Carbohydrate, by difference']

0     1.0
1     1.0
2     1.0
3     1.0
4     1.0
5     1.0
6     1.0
7     1.0
8     1.0
9     1.0
10    1.0
11    1.0
12    1.0
13    1.0
14    1.0
15    1.0
16    1.0
17    1.0
18    1.0
19    1.0
20    1.0
21    1.0
22    1.0
23    1.0
24    1.0
Name: Carbohydrate, by difference, dtype: float64

In [32]:
tmp.columns.unique()

array(['Food_Group', '10:0', '12:0', '13:0', '14:0', '14:1', '15:0',
       '15:1', '16:0', '16:1 c', '16:1 t', '16:1 undifferentiated', '17:0',
       '17:1', '18:0', '18:1 c', '18:1 t', '18:1 undifferentiated',
       '18:1-11t (18:1t n-7)', '18:2 CLAs', '18:2 i', '18:2 n-6 c,c',
       '18:2 t not further defined', '18:2 t,t', '18:2 undifferentiated',
       '18:3 n-3 c,c,c (ALA)', '18:3 n-6 c,c,c', '18:3 undifferentiated',
       '18:3i', '18:4', '20:0', '20:1', '20:2 n-6 c,c', '20:3 n-3',
       '20:3 n-6', '20:3 undifferentiated', '20:4 n-6',
       '20:4 undifferentiated', '20:5 n-3 (EPA)', '21:5', '22:0', '22:1 c',
       '22:1 t', '22:1 undifferentiated', '22:4', '22:5 n-3 (DPA)',
       '22:6 n-3 (DHA)', '24:0', '24:1 c', '4:0', '6:0', '8:0',
       'Adjusted Protein', 'Alanine', 'Alcohol, ethyl', 'Arginine', 'Ash',
       'Aspartic acid', 'Beta-sitosterol', 'Betaine', 'Caffeine',
       'Calcium, Ca', 'Campesterol', 'Carbohydrate, by difference',
       'Carotene, alpha', 'C

In [33]:
tmp['Sucrose'].head(3).T

0    1.0
1    1.0
2    1.0
Name: Sucrose, dtype: float64

In [34]:
tmp.head(10)

Nutrient,Food_Group,10:0,12:0,13:0,14:0,14:1,15:0,15:1,16:0,16:1 c,...,"Vitamin C, total ascorbic acid",Vitamin D,Vitamin D (D2 + D3),Vitamin D2 (ergocalciferol),Vitamin D3 (cholecalciferol),Vitamin E (alpha-tocopherol),"Vitamin E, added",Vitamin K (phylloquinone),Water,"Zinc, Zn"
0,American Indian/Alaska Native Foods,1.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0
1,Baby Foods,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,...,1.0,1.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0
2,Baked Products,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0
3,Beef Products,1.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0
4,Beverages,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0
5,Breakfast Cereals,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
6,Cereal Grains and Pasta,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0
7,Dairy and Egg Products,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
8,Fast Foods,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0
9,Fats and Oils,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0
